In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Make Predict Close price per ten minutes**
1. **Load Data from bitcoin-historical-data**
2. **Using Open price and Volumes to predict Close price**
3. **Re-build our dataframe to fit our purpose(num-2)**
4. **Making dataset by using WindowGenerator to predict time-sequences**
5. **Build a model with TensorFlow(Conv1D and LSTM)**
6. **Lets Predict!**

In [ ]:
!pip install -q tensorflow-gpu
import tensorflow as tf
print("GPU", "Positive" if tf.config.experimental.list_physical_devices("GPU") else "Negative" )
print("version : ",tf.__version__)

In [ ]:
# SET to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# SET data path
path = os.path.join(dirname, filename)

# Load Data
df = pd.read_csv(path)
# Making dataframe clear
df = df.dropna()
print(df.isna().sum())

# I want to use "Open","Volume_(BTC)" and "Volume_(Currency)" as input data(aka features)
# and Set "Close" as y_true

# Drop columns arent used as input data
df.drop(columns=["Timestamp","High","Low", "Weighted_Price"], inplace=True)
print(df)
# Set 'Close' as y_ture
y_true = df.pop('Close')
df['y_true'] = y_true
print(df)

# Split train set and test set
n = len(df)
train_df = df[:int(n*0.9)]
test_df = df[int(n*0.9):]

# Normalize
norm_train_df = (train_df-train_df.mean())/train_df.std()
norm_test_df = (test_df-test_df.mean())/test_df.std()
print(norm_train_df.describe().transpose())

In [ ]:
# Build Window generator
class WindowGenerator():
    # Set Parameters
    def __init__(self,input_width, label_width, offset, 
                 train_df = norm_train_df, test_df = norm_test_df,
                label_columns=None):
        # store data
        self.train_df, self.test_df = train_df, test_df
        # Make label dictionary
        if label_columns is not None:
            self.label_dict = {name: i for i, name in enumerate(label_columns)}
        self.label_columns = label_columns
        
        # Set winodw parameters
        self.input_width, self.label_width = input_width, label_width
        self.total_window_size = input_width + offset
        
        self.input_slice = slice(0,input_width)
        self.input_example = np.arange(self.total_window_size)[self.input_slice]
        
        self.label_start = self.total_window_size - label_width
        self.label_slice = slice(self.label_start,None)
        self.label_example = np.arange(self.total_window_size)[self.label_slice]
        
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input Example: {self.input_example}',
            f'Label Example: {self.label_example}',
            f'Label Name: {self.label_columns}'])
    
    def split_datasets(self, datasets):
        # inputs.shape = (batch, index(time), features)
        # features are in datasets : Open, Volume_(BTC), Volume_(Currency), Close
        # So except Close, the others are used as inputs
        inputs = datasets[:, self.input_slice, :-len(self.label_columns) ]
        labels = datasets[:, self.label_slice, -len(self.label_columns):]
#         if self.label_columns is not None:
#             labels = tf.stack(
#                 [labels[:, :, self.label_dict[name]] for name in self.label_columns],
#             axis=-1)
        # Set Shape
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])
        
        
        return inputs, labels
    
    def datasets_maker(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data, targets=None, sequence_length=self.total_window_size, 
                sequence_stride=1,shuffle=False, batch_size=60)
        ds = ds.map(self.split_datasets)
        
        return ds
    
    @property
    def train(self):
        return self.datasets_maker(self.train_df)
    
    @property
    def test(self):
        return self.datasets_maker(self.test_df)
    
ten = WindowGenerator(input_width=10, label_width=1, offset=10, 
                     label_columns=['Close'])
print("Train : {}\nTest : {}".format(ten.train,ten.test))

In [ ]:
# Build Model
class Build_Model(tf.keras.Model):
    def __init__(self):
        super(Build_Model, self).__init__()
        self.lstm = tf.keras.layers.LSTM(32, return_sequences=False, 
                                        kernel_regularizer='l2')
        self.dense = tf.keras.layers.Dense(32, activation='relu')
        self.last = tf.keras.layers.Dense(1)
        
    def call(self, x):
        x = self.lstm(x)
        x = self.dense(x)
        return self.last(x)

model = Build_Model()

In [ ]:
# Set Loss and Optimizer
loss_object = tf.keras.losses.Huber()
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(x,y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = loss_object(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
@tf.function
def test_step(x,y):
    predictions = model(x)
    loss = loss_object(y, predictions)

In [ ]:
# Get Train and Test datasets
train_dataset = ten.train
test_dataset = ten.test
# Train the model
EPOCHS = 10
for epoch in range(EPOCHS):
    for train_x, train_y in train_dataset:
        train_step(train_x, train_y)
        
    for test_x, test_y in test_dataset:
        test_step(test_x, test_y)
        
    print("EPOCHS : {} ".format(epoch+1))

tf.keras.backend.clear_session()

In [ ]:
# Lets take graphs
import matplotlib.pyplot as plt

origin = norm_test_df['y_true']
forecast = model.predict(test_dataset).ravel(order='C')
print("origin : {}\nforecast : {}".format(origin.shape, forecast.shape))

plt.figure()
plt.plot(range(len(origin)), origin, label="Origin", color='r')
plt.plot(range(len(forecast)), forecast, label="Forecast", color='b')
plt.legend()
plt.show()

tf.keras.backend.clear_session()

# **Finally! We Got Extra Simple prediction model!**